In [54]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import re
import os

In [55]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

df = pd.read_csv('weekly_sales.csv')

In [56]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [57]:
pd.set_option('display.max_columns',None)

In [58]:
df2 = df.filter(['SOLD DATE','ADDRESS','CITY','PRICE','BEDS','BATHS','LOCATION','SQUARE FEET','YEAR BUILT','DAYS ON MARKET','$/SQUARE FEET','LATITUDE','LONGITUDE'])

In [59]:
# df3 = df2.dropna(subset='SOLD DATE')

In [60]:
df2['SOLD_DATE_DT'] = pd.to_datetime(df2['SOLD DATE'])

In [61]:
df2.SOLD_DATE_DT

0     2023-01-20
1     2023-01-20
2     2023-01-20
3     2023-01-20
4     2023-01-20
         ...    
108   2023-01-19
109   2023-01-17
110   2023-01-17
111   2023-01-17
112   2023-01-17
Name: SOLD_DATE_DT, Length: 113, dtype: datetime64[ns]

In [62]:
df2 = df2.loc[(df2['SOLD_DATE_DT'] > '2023-01-14') & (df2['SOLD_DATE_DT'] < '2023-01-22')]

In [63]:
df2.sort_values(by='PRICE',ascending=False)

,SOLD DATE,ADDRESS,CITY,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,LATITUDE,LONGITUDE,SOLD_DATE_DT
25,January-17-2023,6800 Fisher Island Dr #6823,Miami Beach,15500000,4,4.5,FISHER ISLAND,4904.0,2019,NaN,3161.0,25.763482,-80.137699,2023-01-17
26,January-20-2023,1000 S Pointe Dr #3201,Miami Beach,7300000,3,3.5,MURANO AT PORTOFINO CONDO,3365.0,2002,NaN,2169.0,25.768712,-80.137670,2023-01-20
24,January-20-2023,7251 Fisher Island Dr #7251,Miami Beach,5500000,3,3.5,OCEANSIDE @ FISHER ISL CO,3140.0,1992,NaN,1752.0,25.762287,-80.134708,2023-01-20
111,January-17-2023,16901 Collins Ave #2403,Sunny Isles Beach,4200000,3,4.5,JADE SIGNATURE CONDO,2496.0,2018,NaN,1683.0,25.932756,-80.121005,2023-01-17
50,January-19-2023,1581 Brickell Ave Unit T-204,Miami,4050000,4,4.5,VILLA REGINA CONDO,4130.0,1982,NaN,981.0,25.756391,-80.193304,2023-01-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,January-19-2023,3849 NE 169th St #302,North Miami Beach,190000,1,1.5,EDEN ISLES CONDO PART 3,812.0,1970,NaN,234.0,25.936072,-80.133394,2023-01-19
95,January-20-2023,19201 Collins Ave #307,Sunny Isles Beach,180000,0,1.0,THE AVENTURA BEACH CLUB C,420.0,1966,NaN,429.0,25.954474,-80.119740,2023-01-20
10,January-18-2023,950 NW 11th St Unit 1D,Miami,163500,1,1.0,THE TWELVE CONDO,509.0,1975,NaN,321.0,25.783879,-80.210665,2023-01-18
32,January-18-2023,13500 NE 3rd Ct #314,North Miami,128000,1,1.5,Brookview Condo,691.0,1975,NaN,185.0,25.899624,-80.192301,2023-01-18


In [64]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 15th - January 21st')

for i,r in df.iterrows():
    location = (r["LATITUDE"], r["LONGITUDE"])
    info = (r["PRICE"], r["DAYS ON MARKET"], r['$/SQUARE FEET'], r['ADDRESS'], r['CITY'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Price: {bold_start}{info[0]}{bold_end}{new_line} \
    Days on Market: {bold_start}{info[1]}{bold_end}{new_line}Price Per SqFt: \
    {bold_start}{info[2]}{bold_end}{new_line}Address: {bold_start}{info[3]} \
    {bold_end}{new_line}City: {bold_start}{info[4]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [65]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [66]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [67]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_012322
